In [1]:
from cpm.utils import pandas_to_dict
from cpm.generators import Value, Parameters, Wrapper
import cpm.models
import numpy as numpy
import pandas as pd
import ipyparallel as ipp

data = pd.read_csv("simulated_data.csv")

experiment = pandas_to_dict(
    data,
    participant="ppt",
    stimuli="stimulus",
    feedback="reward",
    observed="responses",
    trial_number="trial",
)
sd_start = numpy.random.uniform(0.1, 1, 2) * numpy.array([1, 10])
mean_start = numpy.random.uniform(0, 1, 2) * numpy.array([1, 10])


mean=[0.25, 5]
sd=[0.25, 2.5]

parameters = Parameters(
    # freely varying parameters are indicated by specifying priors
    alpha=Value(
        value=0.5,
        lower=1e-2,
        upper=1,
        prior="truncated_normal",
        args={"mean": mean[0], "sd": sd[0]},
    ),
    beta=Value(
        value=2,
        lower=0,
        upper=10,
        prior="truncated_normal",
        args={"mean": mean[1], "sd": mean[1]},
    ),
    values=numpy.ones(4) / 4,
)

@ipp.require('numpy')
def model_analysis(parameters, trial):
    """
    This function uses the model to estimate latent variables.
    This means that the we use agents choices to estimate the values of the actions.
    """
    # pull out the parameters
    alpha = parameters.alpha
    beta = parameters.beta
    values = numpy.array(
        parameters.values
    )  # copy essentially prevents us from accidentally overwriting the original values
    # pull out the trial information
    stimulus = numpy.array(trial.get("trials"))
    feedback = numpy.array(trial.get("feedback"))
    choice = trial.get("observed")
    choice = choice.astype(int)

    # activate the value of each available action
    # here there are two possible actions, that can take up on 4 different values
    # so we subset the values to only include the ones that are activated...
    # ...according to which stimuli was presented
    activation = values[stimulus - 1]
    # convert the activations to a 2x1 matrix, where rows are actions/outcomes
    activations = activation.reshape(2, 1)
    # calculate a policy based on the activations
    response = cpm.models.decision.Softmax(activations=activations, temperature=beta)
    response.compute()  # compute the policy
    if numpy.isnan(response.policies).any():
        # if the policy is NaN for a given action, then we need to set it to 1
        print(response.policies)
        response.policies[numpy.isnan(response.policies)] = 1
        response.policies = response.policies / numpy.sum(response.policies)
    generated = response.choice()
    # update the value of the chosen action
    mute = numpy.zeros(4)  # mute learning for all cues not presented
    mute[stimulus[choice] - 1] = 1  # unmute the learning for the chosen action
    reward = feedback[choice]  # get the reward of the chosen action
    teacher = numpy.array([reward])
    update = cpm.models.learning.SeparableRule(
        weights=values, feedback=teacher, input=mute, alpha=alpha
    )
    update.compute()
    values = values + update.weights  # update the values
    ## compile output
    output = {
        "policy": response.policies,  # policies
        "stimulus": stimulus,  # stimulus presented
        "response": generated,  # choice based on the policy
        "reward": reward,  # reward of the chosen action
        "values": values[0],  # updated values
        "change": update.weights,  # change in the values
        "activation": activations.flatten(),  # activation of the values
        "dependent": numpy.array([response.policies[1]]),  # dependent variable
    }
    return output

wrapper = Wrapper(model=model_analysis, parameters=parameters, data=data)


from cpm.optimisation import minimise, FminBound
from cpm.optimisation.minimise import LogLikelihood

loss = LogLikelihood.bernoulli


In [2]:

fit = FminBound(
    model=wrapper,  # Wrapper class with the model we specified from before
    data=experiment,  # the data as a list of dictionaries
    minimisation=loss,
    prior=False,
    parallel=True,
    cl=5,  # use all available cores
    libraries=["numpy", "pandas", "cpm"],  # libraries to import on the cluster
    ppt_identifier="ppt",
    number_of_starts=5,
    approx_grad=True,
    pgtol=1e-10,
)

fit.optimise()

Starting optimization 1/5 from [0.53053937 1.55188626]
Starting 5 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/5 [00:00<?, ?engine/s]

Starting optimization 2/5 from [0.01997607 8.81484471]
Starting 5 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/5 [00:00<?, ?engine/s]

Starting optimization 3/5 from [0.04300368 9.64040154]
Starting 5 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/5 [00:00<?, ?engine/s]

Starting optimization 4/5 from [0.94083994 4.68481824]
Starting 5 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/5 [00:00<?, ?engine/s]

Starting optimization 5/5 from [0.47846832 4.24301131]
Starting 5 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/5 [00:00<?, ?engine/s]